In [2]:
import boto3
import pandas as pd
import time
from urllib.parse import urlparse

# --- Configura��es ---
DATABASE = 'default'
TABLE = 'dados_bovespa_refined'
# Certifique-se que este � o local de sa�da de queries configurado no seu Athena
S3_OUTPUT_LOCATION = 's3://aws-athena-query-results-290892317785/' 

# --- Query SQL ---
# Ajuste a data para um dia que voc� tenha processado
query = f"SELECT * FROM \"{DATABASE}\".\"{TABLE}\" WHERE ano=2025 AND mes=7"


# --- Execu��o da Query ---
client = boto3.client('athena', region_name='us-east-1')

response = client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={
        'Database': DATABASE
    },
    ResultConfiguration={
        'OutputLocation': S3_OUTPUT_LOCATION,
    }
)

query_execution_id = response['QueryExecutionId']

# Espera a query terminar
status = 'RUNNING'
while status in ['RUNNING', 'QUEUED']:
    response = client.get_query_execution(QueryExecutionId=query_execution_id)
    status = response['QueryExecution']['Status']['State']
    if status == 'FAILED' or status == 'CANCELLED':
        # Para depura��o, vamos imprimir a raz�o da falha
        error_reason = response['QueryExecution']['Status'].get('StateChangeReason', 'Nenhuma raz�o informada.')
        raise Exception(f'A consulta no Athena falhou! Raz�o: {error_reason}')
    time.sleep(1)

# --- CARREGAMENTO DOS DADOS (M�TODO CORRIGIDO) ---

# 1. Pega o caminho do arquivo de resultado no S3
s3_path = response['QueryExecution']['ResultConfiguration']['OutputLocation']

# 2. Extrai o nome do bucket e o caminho do arquivo (chave)
parsed_url = urlparse(s3_path, allow_fragments=False)
s3_bucket = parsed_url.netloc
s3_key = parsed_url.path.lstrip('/')

# 3. Define um caminho local tempor�rio para salvar o arquivo
local_path = '/tmp/athena_results.csv'

# 4. Usa o boto3 para baixar o arquivo do S3 para o ambiente local do notebook
s3_client = boto3.client('s3', region_name='us-east-1')
s3_client.download_file(s3_bucket, s3_key, local_path)

# 5. Agora o pandas l� o arquivo local, sem precisar da Layer fsspec/s3fs
df = pd.read_csv(local_path)

print("Dados carregados com sucesso do S3 para o ambiente local e lidos pelo Pandas!")
df.head()


Calculation started (calculation_id=d4cc2db8-562c-96c6-38cd-fc3c3a451621) in (session=5ecc2db7-fd66-2eeb-3b93-b060522871c5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Dados carregados com sucesso do S3 para o ambiente local e lidos pelo Pandas!
          asset        type  participacao_percentual  ... mes  dia codigo_acao
0  PORTO SEGURO  ON      NM                    0.457  ...   7   29       PSSA3
1         COSAN  ON      NM                    0.333  ...   7   29       CSAN3
2    GERDAU MET  PN      N1                    0.286  ...   7   29       GOAU4
3        HYPERA  ON      NM                    0.459  ...   7   28       HYPE3
4          PETZ  ON      NM                    0.056  ...   7   29       PETZ3

[5 rows x 11 columns]



In [3]:
# C�lula 2

# Mostra um resumo das colunas, a contagem de valores n�o nulos e os tipos de dados
print("Informa��es do DataFrame:")
df.info()

print("\n" + "="*50 + "\n")

# Mostra estat�sticas descritivas (m�dia, desvio padr�o, etc.) para as colunas num�ricas
print("Estat�sticas Descritivas:")
df.describe()

Calculation started (calculation_id=8ecc2db8-8175-1f08-e894-57b7e4049b7e) in (session=5ecc2db7-fd66-2eeb-3b93-b060522871c5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Informa��es do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   asset                       168 non-null    object 
 1   type                        168 non-null    object 
 2   participacao_percentual     168 non-null    float64
 3   data_coleta                 168 non-null    object 
 4   qtd_teorica                 168 non-null    int64  
 5   timestamp_processamento     168 non-null    object 
 6   lag_processamento_segundos  168 non-null    int64  
 7   ano                         168 non-null    int64  
 8   mes                         168 non-null    int64  
 9   dia                         168 non-null    int64  
 10  codigo_acao                 168 non-null    object 
dtypes: float64(1), int64(5), object(5)
memory usage: 14.6+ KB


Estat�sticas Descritivas:
       partici

In [12]:
import pandas as pd

# Garante que as colunas necess�rias s�o do tipo correto
df['participacao_percentual'] = pd.to_numeric(df['participacao_percentual'])
df['codigo_acao'] = df['codigo_acao'].astype(str)
df['type'] = df['type'].astype(str)

# --- NOVO: Agrupa os dados para consolidar a��es duplicadas ---
# Agrupamos por 'codigo_acao' e 'type' para obter valores �nicos por ativo
# Usamos .agg() para especificar como consolidar as outras colunas
df_agrupado = df.groupby(['codigo_acao', 'type']).agg(
    # Para a participa��o, somamos os valores
    participacao_percentual=('participacao_percentual', 'sum')
).reset_index() # .reset_index() transforma o resultado do groupby de volta em um DataFrame

print("Dados consolidados com sucesso. Removendo duplicatas e somando participa��es.")


# --- AN�LISE 1: TOP 10 A��ES POR PARTICIPA��O (USANDO DADOS AGRUPADOS) ---

# Ordenar e selecionar top 10 a partir do DataFrame j� agrupado
top_10_acoes = df_agrupado.sort_values('participacao_percentual', ascending=False).head(10)

print("\n\nTop 10 A��es por Participa��o Percentual no Ibovespa (Consolidado)")
print("=" * 60)

# Encontrar o valor m�ximo para a escala do gr�fico
max_value_top10 = top_10_acoes['participacao_percentual'].max()

for index, row in top_10_acoes.iterrows():
    # Calcular o comprimento da barra (escala de 0 a 50 caracteres)
    bar_length = int(50 * row['participacao_percentual'] / max_value_top10)
    bar = '�' * bar_length
    print(f"{row['codigo_acao']:<7} | {bar:<50} | {row['participacao_percentual']:.2f}%")

print("=" * 60)


# --- AN�LISE 2: PARTICIPA��O POR TIPO DE A��O (USANDO DADOS AGRUPADOS) ---

# Agrupa por 'type' e soma a participa��o do DataFrame j� consolidado
participacao_por_tipo = df_agrupado.groupby('type')['participacao_percentual'].sum().sort_values(ascending=False).reset_index()

print("\n\nParticipa��o Total por Tipo de A��o (Consolidado)")
print("=" * 60)

# Encontrar o valor m�ximo para a escala do gr�fico
max_value_tipo = participacao_por_tipo['participacao_percentual'].max()

for index, row in participacao_por_tipo.iterrows():
    # Calcular o comprimento da barra
    bar_length = int(50 * row['participacao_percentual'] / max_value_tipo)
    bar = '�' * bar_length
    print(f"{row['type']:<7} | {bar:<50} | {row['participacao_percentual']:.2f}%")

print("=" * 60)

Calculation started (calculation_id=a0cc2dc3-f6e2-c837-bce3-e21a41b6d602) in (session=5ecc2db7-fd66-2eeb-3b93-b060522871c5). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
Dados consolidados com sucesso. Removendo duplicatas e somando participa��es.


Top 10 A��es por Participa��o Percentual no Ibovespa (Consolidado)
VALE3   | �������������������������������������������������� | 23.10%
ITUB4   | �����������������������������������                | 16.17%
PETR4   | �����������������������������                      | 13.77%
PETR3   | ��������������������                               | 9.64%
BBDC4   | ����������������                                   | 7.73%
SBSP3   | ���������������                                    | 7.15%
ELET3   | ��������������                                     | 6.68%
B3SA3   | ��������������                                     | 6.54%
ITSA4   | ������������                                       | 5.91%
ABEV3   | ������������                                       | 5.69%


Participa��o Total por Tipo de A��o (Consolidado)
ON      NM | �������������������������������������������������� | 76.37%
PN      N1 |

## Insights da An�lise

Os gr�ficos de texto acima demonstram a clara concentra��o de peso do Ibovespa em um n�mero limitado de a��es, com destaque para os setores de commodities e financeiro.

Adicionalmente, a an�lise por tipo de ativo mostra a domin�ncia de A��es Ordin�rias (ON) e Preferenciais (PN) na composi��o do �ndice, refletindo a estrutura de capital das maiores empresas listadas no Brasil.